In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd


In [5]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.nunique()
application_type_counts = application_df["APPLICATION_TYPE"].value_counts().to_dict()
cutoff_value = 527
application_types_to_replace = []
for atype in application_type_counts:
    if application_type_counts[atype] < cutoff_value:
        application_types_to_replace.append(atype)
# Replace in dataframe
# After creating the application_types_to_replace list,
# another for loop is used to replace the application types in the "APPLICATION_TYPE" column of the DataFrame with "Other".
# This ensures that the application types identified in the application_types_to_replace list are replaced accordingly.
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()
classifications_counts = application_df["CLASSIFICATION"].value_counts().to_dict()
cutoff_value_c = 1882
classifications_to_replace = []
for classf in classifications_counts:
    if classifications_counts[classf] < cutoff_value_c:
        classifications_to_replace.append(classf)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


dummy_df = pd.get_dummies(application_df[categorical_columns], columns=categorical_columns)



application_df.drop(columns=categorical_columns, inplace=True)

application_df = pd.concat([application_df, dummy_df], axis=1)



Attempt 1:

In [7]:
def detect_columns_with_outliers_iqr(df, lower_percentile=25, upper_percentile=75, multiplier=1.5):
    # Initialize an empty list to store column names with outliers
    columns_with_outliers = []

    # Loop through each column in the DataFrame
    for column_name in df.columns:
        # Calculate the IQR for the column
        Q1 = df[column_name].quantile(lower_percentile / 100)
        Q3 = df[column_name].quantile(upper_percentile / 100)
        IQR = Q3 - Q1

        # Define the lower and upper bounds to identify outliers
        lower_bound = Q1 - multiplier * IQR
        upper_bound = Q3 + multiplier * IQR

        # Check if any value is outside the bounds
        if any((df[column_name] < lower_bound) | (df[column_name] > upper_bound)):
            columns_with_outliers.append(column_name)

    return columns_with_outliers


def drop_outliers_iqr(df, column_name, lower_percentile=25, upper_percentile=75, multiplier=1.5):
    # Calculate the IQR for the specified column
    Q1 = df[column_name].quantile(lower_percentile / 100)
    Q3 = df[column_name].quantile(upper_percentile / 100)
    IQR = Q3 - Q1

    # Define the lower and upper bounds to identify outliers
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR

    # Drop rows with values outside the bounds
    df_cleaned = df[(df[column_name] >= lower_bound) & (df[column_name] <= upper_bound)]

    return df_cleaned



In [11]:
y=application_df['IS_SUCCESSFUL']
x = application_df.drop(columns=['IS_SUCCESSFUL'])
print(x.shape)
columns_with_outliers = detect_columns_with_outliers_iqr(x, lower_percentile=25, upper_percentile=75, multiplier=1.5)
print("Columns with outliers based on IQR:", columns_with_outliers)
for c in columns_with_outliers:
  x=drop_outliers_iqr(x,c,lower_percentile=25, upper_percentile=75, multiplier=1.5)
print(x.shape)

(34299, 43)
Columns with outliers based on IQR: ['STATUS', 'ASK_AMT', 'APPLICATION_TYPE_Other', 'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3', 'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6', 'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8', 'AFFILIATION_Family/Parent', 'AFFILIATION_National', 'AFFILIATION_Other', 'AFFILIATION_Regional', 'CLASSIFICATION_C1200', 'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000', 'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare', 'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev', 'ORGANIZATION_Co-operative', 'ORGANIZATION_Corporation', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999', 'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M', 'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M', 'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y']
(11478, 43)


In [12]:

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
new_nn = tf.keras.models.Sequential()

# Input layer
new_nn.add(tf.keras.layers.InputLayer(input_shape=(X_train_scaled.shape[1],)))

# First hidden layer with 64 neurons and ReLU activation
new_nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Dropout layer to reduce overfitting
new_nn.add(tf.keras.layers.Dropout(0.2))

# Second hidden layer with 32 neurons and ReLU activation
new_nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Third hidden layer with 16 neurons and ReLU activation
new_nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Fourth hidden layer with 8 neurons and ReLU activation
new_nn.add(tf.keras.layers.Dense(units=8, activation='relu'))

# Output layer with 1 neuron and sigmoid activation for binary classification
new_nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the new model
new_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Check the summary of the new model
new_nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                2816      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                                 
 dense_3 (Dense)             (None, 8)                 136       
                                                                 
 dense_4 (Dense)             (None, 1)                 9         
                                                                 
Total params: 5,569
Trainable params: 5,569
Non-trainabl

In [15]:
num_epochs=150
batch_size=32
new_nn.fit(X_train_scaled, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_scaled, y_test))

Epoch 1/150
858/858 [==============================] - 5s 3ms/step - loss: 0.5846 - accuracy: 0.7112 - val_loss: 0.5670 - val_accuracy: 0.7223
Epoch 2/150
858/858 [==============================] - 2s 3ms/step - loss: 0.5631 - accuracy: 0.7266 - val_loss: 0.5611 - val_accuracy: 0.7233
Epoch 3/150
858/858 [==============================] - 2s 3ms/step - loss: 0.5581 - accuracy: 0.7284 - val_loss: 0.5577 - val_accuracy: 0.7265
Epoch 4/150
858/858 [==============================] - 3s 3ms/step - loss: 0.5558 - accuracy: 0.7281 - val_loss: 0.5581 - val_accuracy: 0.7257
Epoch 5/150
858/858 [==============================] - 2s 3ms/step - loss: 0.5523 - accuracy: 0.7292 - val_loss: 0.5633 - val_accuracy: 0.7248
Epoch 6/150
858/858 [==============================] - 2s 3ms/step - loss: 0.5521 - accuracy: 0.7307 - val_loss: 0.5583 - val_accuracy: 0.7259
Epoch 7/150
858/858 [==============================] - 2s 3ms/step - loss: 0.5518 - accuracy: 0.7316 - val_loss: 0.5582 - val_accuracy: 0.7273

In [17]:
model_loss, model_accuracy = new_nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 1s - loss: 0.5640 - accuracy: 0.7245 - 576ms/epoch - 3ms/step
Loss: 0.5640274286270142, Accuracy: 0.7244898080825806


Attemp 2

In [18]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.nunique()
application_type_counts = application_df["APPLICATION_TYPE"].value_counts().to_dict()
cutoff_value = 100
application_types_to_replace = []
for atype in application_type_counts:
    if application_type_counts[atype] < cutoff_value:
        application_types_to_replace.append(atype)
# Replace in dataframe
# After creating the application_types_to_replace list,
# another for loop is used to replace the application types in the "APPLICATION_TYPE" column of the DataFrame with "Other".
# This ensures that the application types identified in the application_types_to_replace list are replaced accordingly.
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()
classifications_counts = application_df["CLASSIFICATION"].value_counts().to_dict()
cutoff_value_c = 100
classifications_to_replace = []
for classf in classifications_counts:
    if classifications_counts[classf] < cutoff_value_c:
        classifications_to_replace.append(classf)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


dummy_df = pd.get_dummies(application_df[categorical_columns], columns=categorical_columns)



application_df.drop(columns=categorical_columns, inplace=True)

application_df = pd.concat([application_df, dummy_df], axis=1)


In [19]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
new_nn = tf.keras.models.Sequential()

# Input layer
new_nn.add(tf.keras.layers.InputLayer(input_shape=(X_train_scaled.shape[1],)))

# First hidden layer with 64 neurons and ReLU activation
new_nn.add(tf.keras.layers.Dense(units=64, activation='relu'))

# Dropout layer to reduce overfitting
new_nn.add(tf.keras.layers.Dropout(0.2))

# Second hidden layer with 32 neurons and ReLU activation
new_nn.add(tf.keras.layers.Dense(units=32, activation='relu'))

# Third hidden layer with 16 neurons and ReLU activation
new_nn.add(tf.keras.layers.Dense(units=16, activation='relu'))

# Fourth hidden layer with 8 neurons and ReLU activation
new_nn.add(tf.keras.layers.Dense(units=8, activation='relu'))

# Output layer with 1 neuron and sigmoid activation for binary classification
new_nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the new model
new_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Check the summary of the new model
new_nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 64)                2816      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_6 (Dense)             (None, 32)                2080      
                                                                 
 dense_7 (Dense)             (None, 16)                528       
                                                                 
 dense_8 (Dense)             (None, 8)                 136       
                                                                 
 dense_9 (Dense)             (None, 1)                 9         
                                                                 
Total params: 5,569
Trainable params: 5,569
Non-traina

In [21]:
num_epochs=150
batch_size=32
new_nn.fit(X_train_scaled, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_scaled, y_test))

Epoch 1/150
858/858 [==============================] - 5s 3ms/step - loss: 0.5818 - accuracy: 0.7107 - val_loss: 0.5654 - val_accuracy: 0.7223
Epoch 2/150
858/858 [==============================] - 2s 3ms/step - loss: 0.5584 - accuracy: 0.7273 - val_loss: 0.5626 - val_accuracy: 0.7229
Epoch 3/150
858/858 [==============================] - 2s 3ms/step - loss: 0.5573 - accuracy: 0.7266 - val_loss: 0.5624 - val_accuracy: 0.7233
Epoch 4/150
858/858 [==============================] - 2s 3ms/step - loss: 0.5545 - accuracy: 0.7292 - val_loss: 0.5594 - val_accuracy: 0.7204
Epoch 5/150
858/858 [==============================] - 3s 3ms/step - loss: 0.5533 - accuracy: 0.7298 - val_loss: 0.5581 - val_accuracy: 0.7251
Epoch 6/150
858/858 [==============================] - 3s 3ms/step - loss: 0.5513 - accuracy: 0.7301 - val_loss: 0.5584 - val_accuracy: 0.7280
Epoch 7/150
858/858 [==============================] - 2s 2ms/step - loss: 0.5502 - accuracy: 0.7295 - val_loss: 0.5594 - val_accuracy: 0.7287

In [22]:
model_loss, model_accuracy = new_nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5622 - accuracy: 0.7265 - 309ms/epoch - 1ms/step
Loss: 0.5622490644454956, Accuracy: 0.7265306115150452


Attempt 3:

In [24]:
sophisticated_nn = tf.keras.models.Sequential()

sophisticated_nn.add(tf.keras.layers.InputLayer(input_shape=(X_train_scaled.shape[1],)))


sophisticated_nn.add(tf.keras.layers.Dense(units=128, activation='relu'))


sophisticated_nn.add(tf.keras.layers.Dropout(0.2))


sophisticated_nn.add(tf.keras.layers.Dense(units=64, activation='relu'))


sophisticated_nn.add(tf.keras.layers.Dense(units=32, activation='relu'))


sophisticated_nn.add(tf.keras.layers.Dense(units=16, activation='relu'))


sophisticated_nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


sophisticated_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [25]:
num_epochs=50
batch_size=32
sophisticated_nn.fit(X_train_scaled, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_scaled, y_test))

Epoch 1/50
858/858 [==============================] - 6s 4ms/step - loss: 0.5707 - accuracy: 0.7200 - val_loss: 0.5649 - val_accuracy: 0.7276
Epoch 2/50
858/858 [==============================] - 3s 3ms/step - loss: 0.5567 - accuracy: 0.7291 - val_loss: 0.5622 - val_accuracy: 0.7227
Epoch 3/50
858/858 [==============================] - 2s 3ms/step - loss: 0.5534 - accuracy: 0.7296 - val_loss: 0.5602 - val_accuracy: 0.7246
Epoch 4/50
858/858 [==============================] - 2s 3ms/step - loss: 0.5524 - accuracy: 0.7295 - val_loss: 0.5568 - val_accuracy: 0.7286
Epoch 5/50
858/858 [==============================] - 3s 3ms/step - loss: 0.5507 - accuracy: 0.7311 - val_loss: 0.5594 - val_accuracy: 0.7214
Epoch 6/50
858/858 [==============================] - 3s 3ms/step - loss: 0.5499 - accuracy: 0.7315 - val_loss: 0.5606 - val_accuracy: 0.7255
Epoch 7/50
858/858 [==============================] - 2s 3ms/step - loss: 0.5490 - accuracy: 0.7329 - val_loss: 0.5589 - val_accuracy: 0.7268
Epoch 

In [26]:
model_loss, model_accuracy = sophisticated_nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5615 - accuracy: 0.7252 - 467ms/epoch - 2ms/step
Loss: 0.5614736676216125, Accuracy: 0.725218653678894


Attempt 4:

In [28]:
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df = application_df.drop(columns=["EIN", "NAME"])
application_df.nunique()
application_type_counts = application_df["APPLICATION_TYPE"].value_counts().to_dict()
cutoff_value = 527
application_types_to_replace = []
for atype in application_type_counts:
    if application_type_counts[atype] < cutoff_value:
        application_types_to_replace.append(atype)
# Replace in dataframe
# After creating the application_types_to_replace list,
# another for loop is used to replace the application types in the "APPLICATION_TYPE" column of the DataFrame with "Other".
# This ensures that the application types identified in the application_types_to_replace list are replaced accordingly.
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()
classifications_counts = application_df["CLASSIFICATION"].value_counts().to_dict()
cutoff_value_c = 1882
classifications_to_replace = []
for classf in classifications_counts:
    if classifications_counts[classf] < cutoff_value_c:
        classifications_to_replace.append(classf)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")


categorical_columns = ['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS']


dummy_df = pd.get_dummies(application_df[categorical_columns], columns=categorical_columns)



application_df.drop(columns=categorical_columns, inplace=True)

application_df = pd.concat([application_df, dummy_df], axis=1)


In [29]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
sophisticated_nn = tf.keras.models.Sequential()

sophisticated_nn.add(tf.keras.layers.InputLayer(input_shape=(X_train_scaled.shape[1],)))


sophisticated_nn.add(tf.keras.layers.Dense(units=128, activation='relu'))


sophisticated_nn.add(tf.keras.layers.Dropout(0.2))


sophisticated_nn.add(tf.keras.layers.Dense(units=64, activation='relu'))


sophisticated_nn.add(tf.keras.layers.Dense(units=32, activation='relu'))


sophisticated_nn.add(tf.keras.layers.Dense(units=16, activation='relu'))


sophisticated_nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


sophisticated_nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [31]:
num_epochs=80
batch_size=32
sophisticated_nn.fit(X_train_scaled, y_train, epochs=num_epochs, batch_size=batch_size, validation_data=(X_test_scaled, y_test))

Epoch 1/80
858/858 [==============================] - 4s 3ms/step - loss: 0.5714 - accuracy: 0.7197 - val_loss: 0.5633 - val_accuracy: 0.7227
Epoch 2/80
858/858 [==============================] - 2s 3ms/step - loss: 0.5562 - accuracy: 0.7293 - val_loss: 0.5611 - val_accuracy: 0.7200
Epoch 3/80
858/858 [==============================] - 4s 4ms/step - loss: 0.5539 - accuracy: 0.7304 - val_loss: 0.5605 - val_accuracy: 0.7273
Epoch 4/80
858/858 [==============================] - 2s 3ms/step - loss: 0.5524 - accuracy: 0.7306 - val_loss: 0.5632 - val_accuracy: 0.7270
Epoch 5/80
858/858 [==============================] - 2s 3ms/step - loss: 0.5509 - accuracy: 0.7310 - val_loss: 0.5583 - val_accuracy: 0.7265
Epoch 6/80
858/858 [==============================] - 2s 3ms/step - loss: 0.5487 - accuracy: 0.7329 - val_loss: 0.5609 - val_accuracy: 0.7286
Epoch 7/80
858/858 [==============================] - 4s 4ms/step - loss: 0.5485 - accuracy: 0.7325 - val_loss: 0.5563 - val_accuracy: 0.7284
Epoch 

In [32]:
model_loss, model_accuracy = sophisticated_nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - loss: 0.5636 - accuracy: 0.7259 - 454ms/epoch - 2ms/step
Loss: 0.5636117458343506, Accuracy: 0.7259474992752075
